In [3]:
import os
import random
import numpy as np
import tensorflow as tf

# Imposta il seed globale
seed = 0
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

#from tensorflow.keras.constraints import MinMaxNorm
from tensorflow.keras.callbacks import ReduceLROnPlateau

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Normalization, Dense, Dropout, ReLU, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import LearningRateScheduler, TensorBoard, ProgbarLogger
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import h5py
import sys
import importlib
import time

# Imposta float32 come tipo predefinito in Keras
tf.keras.backend.set_floatx('float32')

# Tipo da usare esplicitamente (es. in numpy)
force_datatype = np.float32

use_gpu = 1

# ------------------ Configurazione GPU ------------------ #
if not use_gpu:
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Disabilita la GPU

print(f"Using GPU: {tf.config.list_physical_devices('GPU')}")
print(tf.config.list_physical_devices('GPU'))

tf.config.optimizer.set_jit(False)  # XLA off


2025-04-23 13:45:15.161941: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745408715.247721    1202 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745408715.271610    1202 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745408715.430575    1202 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745408715.430601    1202 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745408715.430604    1202 computation_placer.cc:177] computation placer alr

Using GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Define variables

In [ ]:
base_folder = '/mnt/c/Users/Work/Desktop/deepMIMO/RIS/DeepMIMOv1-LIS-DeepLearning-Taha/'
input_folder = base_folder + 'Output Matlab/'

#DeepMIMO_dataset_folder = input_folder + 'DeepMIMO Dataset/'
DL_dataset_folder = input_folder + 'DL Dataset/'
network_folder_in = input_folder + 'Neural Network/'

output_folder = base_folder + 'Output_Python/'
network_folder_out = output_folder + 'Neural_Network/'
network_folder_out_YPredicted = output_folder + 'Neural Network/YPredicted/'
saved_models = network_folder_out + 'saved_models/'
figure_folder = output_folder + 'Figures/'

ris=0

My_ar = [32, 64]
Mz_ar = [32, 64]
Mx = 1
My = My_ar[ris]
Mz = Mz_ar[ris]
print('RIS ', My, Mz)

M_bar=8
Ur_rows = [1000, 1200]
#              0    1      2      3      4      5      6
Training_Size=[2, 10000, 14000, 18000, 22000, 26000, 30000]
#print(Training_Size)
Training_Size_dd = Training_Size[1]
print("Training_Size_dd:", Training_Size_dd)

RIS  32 32
Training_Size_dd: 10000


## Directly import XTrain

## Load Dataset DL_input_reshaped

In [5]:
filename_DL_input_reshaped = DL_dataset_folder + 'DL_input_reshaped' + '_seed' + str(seed) + '_grid' + str(Ur_rows[1]) + '_M' + str(My) + str(Mz) + '_Mbar' + str(M_bar) + '.mat'
filename_DL_output_reshaped = DL_dataset_folder + 'DL_output_reshaped' + '_seed' + str(seed) + '_grid' + str(Ur_rows[1]) + '_M' + str(My) + str(Mz) + '_Mbar' + str(M_bar) + '.mat'
filename_RandP_all = DL_dataset_folder + 'RandP_all' + '_seed' + str(seed) + '_grid' + str(Ur_rows[1]) + '_M' + str(My) + str(Mz) + '_Mbar' + str(M_bar) + '.mat'

# Load the data using h5py for MATLAB v7.3 files
with h5py.File(filename_DL_input_reshaped, 'r') as f:
    DL_input_reshaped = np.array(f['DL_input_reshaped'][:], dtype=force_datatype)
with h5py.File(filename_DL_output_reshaped, 'r') as f:
    DL_output_reshaped = np.array(f['DL_output_reshaped'][:], dtype=force_datatype)
with h5py.File(filename_RandP_all, 'r') as f:
    RandP_all = np.array(f['RandP_all'][:], dtype=force_datatype)

print(DL_input_reshaped.shape)
print(DL_output_reshaped.shape)
print(RandP_all.shape)

print(np.min(DL_input_reshaped))
print(np.max(DL_input_reshaped))
print(np.min(DL_output_reshaped))
print(np.max(DL_output_reshaped))

(36200, 1, 1, 1024)
(36200, 1024, 1, 1)
(1, 36200)
-0.9966719
0.99682105
0.0
1.0


## Load Rates

In [6]:
# Costruzione del nome file
filename_DL_output_un_reshaped = DL_dataset_folder + 'DL_output_un_reshaped' + '_seed' + str(seed) + '_grid' + str(Ur_rows[1]) + '_M' + str(My) + str(Mz) + '_Mbar' + str(M_bar) + '.mat'

# Load the data using h5py for MATLAB v7.3 files
with h5py.File(filename_DL_output_un_reshaped, 'r') as f:
    # Accesso alla variabile (nome del dataset = nome della variabile in MATLAB)
    YValidation_un = np.array(f['DL_output_un_reshaped'], dtype=force_datatype)

print(YValidation_un.shape)

YValidation_un2 = np.transpose(YValidation_un, (3, 2, 1, 0))  # conversione a (b, z, y, x)
print(YValidation_un2.shape)

(6200, 1024, 1, 1)
(1, 1, 1024, 6200)


## Dataset split originale

In [7]:
# Flatten the input and output arrays if necessary
#X = DL_input_reshaped.reshape(DL_input_reshaped.shape[0], -1).astype(np.float32)
#Y = DL_output_reshaped.reshape(DL_output_reshaped.shape[0], -1).astype(np.float32)

# Split the dataset into training and validation sets
#X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=validation_size / (training_size + validation_size), shuffle=False, random_state=seed)

RandP_all2 = np.squeeze(np.array(RandP_all.astype(int))) - 1

Training_Ind = RandP_all2[0:Training_Size_dd]

Validation_Size = 6200
Validation_Ind = RandP_all2[-Validation_Size:]

print(Training_Ind.shape)
print(Validation_Ind.shape)

X_train = np.array(DL_input_reshaped[Training_Ind, :, :, :], dtype=force_datatype).squeeze()
Y_train = np.array(DL_output_reshaped[Training_Ind, :, :, :], dtype=force_datatype).squeeze()
X_val = np.array(DL_input_reshaped[Validation_Ind, :, :, :], dtype=force_datatype).squeeze()
Y_val = np.array(DL_output_reshaped[Validation_Ind, :, :, :], dtype=force_datatype).squeeze()

print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)

(10000,)
(6200,)
(10000, 1024)
(10000, 1024)
(6200, 1024)
(6200, 1024)


## Load Matlab trainedNet model

In [8]:
# Costruisci il path e il nome del modulo
modulename = 'trainedNet' + '_seed' + str(seed) + '_grid' + str(Ur_rows[1]) + '_M' + str(My) + str(Mz) + '_Mbar' + str(M_bar) + '_' + str(Training_Size_dd)
print(modulename)

# Aggiungi la cartella al path (quella che contiene la cartella del modello)
sys.path.append(network_folder_in)
filename_module = os.path.join(network_folder_in, modulename)

# Cambia la working directory temporaneamente
old_cwd = os.getcwd()
os.chdir(network_folder_in)

# Importa il modulo ed esegui il load
myModel = importlib.import_module(modulename)
#import model as myModel
model_mat = myModel.load_model()

# Ripristina la working directory originale
os.chdir(old_cwd)

# Ora puoi usare il modello
model_mat.summary()

print(model_mat.layers[1].mean.shape) # layers[1] is the Normalization layer
print(np.array(model_mat.layers[1].mean))

print(model_mat.layers[3].dtype)


print(model_mat.layers[1].mean.shape)
print(model_mat.layers[1].variance.shape)
print("Sono 1024 valori di mean tutti uguali, 1024 valori di varianza tutti uguali e 1 epsilon per stabilità numerica.")

trainedNet_seed0_grid1200_M3232_Mbar8_10000


I0000 00:00:1745408723.848135    1202 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13709 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:47:00.0, compute capability: 8.6


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_unnormalized (InputLayer) │ (None, 1024, 1, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ input_ (Normalization)          │ (None, 1024, 1, 1)     │         2,049 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully1_preFlatten1 (Reshape)    │ (None, 1, 1, 1024)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully1_ (Dense)                 │ (None, 1, 1, 1024)     │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 1, 1, 1024)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1, 1, 1024)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully2_preFlatten1 (Reshape)    │ (None, 1, 1, 1024)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully2_ (Dense)                 │ (None, 1, 1, 4096)     │     4,198,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 1, 1, 4096)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1, 1, 4096)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully3_preFlatten1 (Reshape)    │ (None, 1, 1, 4096)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully3_ (Dense)                 │ (None, 1, 1, 4096)     │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 1, 1, 4096)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1, 1, 4096)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully4_preFlatten1 (Reshape)    │ (None, 1, 1, 4096)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully4_ (Dense)                 │ (None, 1, 1, 1024)     │     4,195,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,226,689 (100.05 MB)

 Trainable params: 26,224,640 (100.04 MB)

 Non-trainable params: 2,049 (8.01 KB)

(1, 1024, 1, 1)
[[[[0.00030797]]

  [[0.00030797]]

  [[0.00030797]]

  ...

  [[0.00030797]]

  [[0.00030797]]

  [[0.00030797]]]]
float32
(1, 1024, 1, 1)
(1, 1024, 1, 1)
Sono 1024 valori di mean tutti uguali, 1024 valori di varianza tutti uguali e 1 epsilon per stabilità numerica.


## Prediction (!!! Before running, check DL_input_reshap concat comment)

## Recreate the same network in Python

In [ ]:
# Flatten the input and output arrays if necessary
#X = DL_input_reshaped.reshape(DL_input_reshaped.shape[0], -1).astype(np.float32)
#Y = DL_output_reshaped.reshape(DL_output_reshaped.shape[0], -1).astype(np.float32)

# Split the dataset into training and validation sets
#X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=validation_size / (training_size + validation_size), shuffle=False, random_state=seed)
#XTrain = np.array(DL_input_reshaped[:, 0, 0, Training_Ind], dtype=np.float32)
#YTrain = np.array(DL_output_reshaped[0, 0, :, Training_Ind], dtype=np.float32)
#XValidation = np.array(DL_input_reshaped[:, 0, 0, Validation_Ind], dtype=np.float32)
#YValidation = np.array(DL_output_reshaped[0, 0, :, Validation_Ind], dtype=np.float32)
#


### Normalize the training data (zero-center normalization) with Scikit-learn

✅ 1. Normalization Layer (Keras)

✔️ Pro
- Incluso nel modello: la normalizzazione fa parte della rete, quindi viene salvata nel .h5 o .pb.

- Usato sia in training che in inference, senza fare altro.

- Funziona bene anche con serving, export in TensorFlow Lite, TensorFlow.js, ecc.

- Può essere adattato con .adapt() usando un tf.data.Dataset o numpy.

❌ Contro
- Supporta solo alcune strategie di normalizzazione (es. mean-std su ogni feature).

- Meno flessibile se hai bisogno di strategie avanzate (e.g. quantile scaling, robust scaling).

✅ 2. StandardScaler (Scikit-learn)

✔️ Pro
- Più flessibile: puoi usare altri scaler (MinMaxScaler, RobustScaler, ecc.).

- Ideale se il preprocessing viene gestito fuori dal modello (es. in un pipeline scikit-learn, o se non stai usando solo TensorFlow).

❌ Contro
- La normalizzazione non è inclusa nel modello.

- Devi salvare il scaler a parte e applicarlo manualmente in fase di inference.

- Rischio di mismatch tra training e inference se dimentichi qualcosa (es. diverse medie).

In [ ]:
normalized = 0

### Load normalization parameters from Matlab trained model


In [9]:
filename_trainedNet_scaler = network_folder_in + 'trainedNet_scaler' + '_seed' + str(seed) + '_grid' + str(Ur_rows[1]) + '_M' + str(My) + str(Mz) + '_Mbar' + str(M_bar) + '_' + str(Training_Size_dd) + '.mat'

with h5py.File(filename_trainedNet_scaler, 'r') as f:
    trainedNet_scaler = f['trainedNet_scaler'][:][0][0]

print(trainedNet_scaler.shape)
print(trainedNet_scaler) # should be -5.1904644e-06 for Training_Size_dd=30000


# This layer will shift and scale inputs into a distribution centered around 0 with standard deviation 1. 
# It accomplishes this by precomputing the mean and variance of the data, and calling (input - mean) / sqrt(var) at runtime.
# The mean and variance values for the layer must be either supplied on construction or learned via adapt(). 
# adapt() will compute the mean and variance of the data and store them as the layer's weights. 
# adapt() should be called before fit(), evaluate(), or predict().

mean_array = np.array([trainedNet_scaler]*X_train.shape[1], dtype=force_datatype)
variance_array =  np.array([1]*X_train.shape[1], dtype=force_datatype)
print(mean_array.shape)
print(mean_array[0])
print(variance_array.shape)
print(variance_array[0])

normalizer = Normalization(axis=1, mean=mean_array, variance=variance_array, name='normalization')

print(normalizer.input_mean)
print(normalizer.input_variance)

normalized = 1

()
0.00030796975
(1024,)
0.00030796975
(1024,)
1.0
[0.00030797 0.00030797 0.00030797 ... 0.00030797 0.00030797 0.00030797]
[1. 1. 1. ... 1. 1. 1.]


### Compute normalization parameters from Training Dataset


In [ ]:
#normalizer = Normalization(axis=3, name='normalization')
#normalizer.adapt(X_train)   # adapt è richiesto da Normalization prima di fit, ma dopo aver costriuito il modello
# ma poichè vogliamo solo la mean e varianza = 1, non possiamo usarlo

# Possiamo ovviare con questo trucco
mean_array = np.array([np.mean(np.mean(X_train, axis=1))]*X_train.shape[1], dtype=force_datatype)
variance_array =  np.array([1]*X_train.shape[1], dtype=force_datatype)
print(mean_array.shape)
print(mean_array[0])
print(variance_array.shape)
print(variance_array[0])

normalizer = Normalization(axis=1, mean=mean_array, variance=variance_array, name='normalization')

m = normalizer.input_mean
v = normalizer.input_variance

print(m.shape)
print(m[0])
print(v.shape)
print(v[0])

normalized = 1

### Normalize data

In [10]:
# Normalizzazione manuale se già hai mean_array e variance_array da MATLAB
X_train_normalized = np.array((X_train - mean_array) / np.sqrt(variance_array), dtype=force_datatype)

X_val_normalized = np.array((X_val - mean_array) / np.sqrt(variance_array), dtype=force_datatype)

print(mean_array)
print(variance_array)

normalized = 1

[0.00030797 0.00030797 0.00030797 ... 0.00030797 0.00030797 0.00030797]
[1. 1. 1. ... 1. 1. 1.]


In [11]:
print(X_train[0][0:5])
print(mean_array[0:5])
print(X_train_normalized[0][0:5])

[-0.3836985   0.22742581 -0.14068858  0.42388362 -0.42758086]
[0.00030797 0.00030797 0.00030797 0.00030797 0.00030797]
[-0.38400647  0.22711784 -0.14099655  0.42357564 -0.42788884]


## DL Model Definition

In [12]:
if normalized == 1:
    xt = X_train_normalized
    xv = X_val_normalized
else:
    xt = X_train
    xv = X_val

In [40]:
# Define the neural network architecture
model_py = Sequential([
    Input(shape=(X_train.shape[1],), name='input'),

    Dense(units=Y_train.shape[1], kernel_regularizer=l2(1e-4), name='Fully1_'),
    ReLU(name='relu1'),
    Dropout(0.5, name='dropout1'),

    Dense(units=4 * Y_train.shape[1], kernel_regularizer=l2(1e-4), name='Fully2_'),
    ReLU(name='relu2'),
    Dropout(0.5, name='dropout2'),

    Dense(units=4 * Y_train.shape[1], kernel_regularizer=l2(1e-4), name='Fully3_'),
    ReLU(name='relu3'),
    Dropout(0.5, name='dropout3'),

    Dense(units=Y_train.shape[1], kernel_regularizer=l2(1e-4), name='Fully4_'),
])

# Compile the model with SGD optimizer and mean squared error loss
optimizer = SGD(learning_rate=1e-1, momentum=0.9)

def mse_keras(y_true, y_pred): # Not working
    squared_error = tf.square(y_true - y_pred)  # shape: (batch_size, output_dim)=6200,1024
    loss = tf.reduce_mean(squared_error) # scalar
    return loss

def mse_custom(y_true, y_pred):
    # Calcola l'errore quadratico tra vero e predetto
    squared_error = tf.square(y_true - y_pred)  # shape: (batch_size, output_dim)=6200,1024

    # Somma degli errori lungo l'ultima dimensione (output_dim)
    sum_squared_error = tf.reduce_sum(squared_error, axis=-1)  # shape: (batch_size,)=6200

    # Media su tutto il batch
    loss = 0.5 * tf.reduce_mean(sum_squared_error)  # scalar
    return loss

def mse_matlab(y_true, y_pred): # Not working
    # A regression layer computes the half-mean-squared-error loss for regression tasks:
    # https://www.mathworks.com/help//releases/R2021a/deeplearning/ref/regressionlayer.html?searchHighlight=regressionLayer&searchResultIndex=1
    squared_error = tf.square(y_true - y_pred)  # shape: (batch_size, output_dim)=6200,1024
    
    loss = 0.5 * tf.reduce_mean(squared_error)
    return loss

#model_py.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mse'])
#model_py.compile(optimizer=optimizer, loss=mse_keras, metrics=['mse'])
#model_py.compile(optimizer=optimizer, loss=mse_matlab, metrics=['mse'])
model_py.compile(optimizer=optimizer, loss=mse_custom, metrics=['mse'])
model_py.summary()

print(model_py.loss)

# ------------------ Learning Rate Scheduler ------------------ #
def lr_schedule(epoch, lr):
    if epoch > 0 and epoch % 5 == 0: # Prima era modulo 3
        return lr * 0.5  # Drop learning rate by factor of 0.5 every x epochs
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,         # Riduce di metà
    patience=2,         # Numero di epoche senza miglioramento ≥ y
    min_delta=0.1,        # Miglioramento minimo da considerare significativo
    verbose=1
)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Fully1_ (Dense)                 │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu1 (ReLU)                    │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout1 (Dropout)              │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully2_ (Dense)                 │ (None, 4096)           │     4,198,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu2 (ReLU)                    │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout2 (Dropout)              │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully3_ (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu3 (ReLU)                    │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout3 (Dropout)              │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Fully4_ (Dense)                 │ (None, 1024)           │     4,195,328 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,224,640 (100.04 MB)

 Trainable params: 26,224,640 (100.04 MB)

 Non-trainable params: 0 (0.00 B)

<function mse_custom at 0x7fdc8c2f34c0>


In [41]:
if normalized == 0:

    # adapt() funziona solo se il layer di normalizzazione non è stato inizializzato con mean e variance

    # Supponiamo che X_train abbia shape (samples, 1024, 1, 1)
    normalizer_layer = model_py.layers[0]
    normalizer_layer.adapt(tf.convert_to_tensor(xt, dtype=tf.float32))  # Normalizzazione su training data

    m = np.array(model_py.layers[0].mean)
    v = np.array(model_py.layers[0].variance)

    print(m.shape)
    print(np.max(m))
    print(np.min(m))

    print(v.shape)
    print(np.max(v))
    print(np.min(v))


## DL Model Training

In [42]:
model_type = 'original12-0.5-new-'
tensorboard_logs = network_folder_out + 'tensorboard_logs/'+model_type

# ------------------ Callback ------------------ #
tensorboard_callback = TensorBoard(log_dir=tensorboard_logs, histogram_freq=1)

In [43]:
%load_ext tensorboard
#!tensorboad --logdir /mnt/c/Users/Work/Desktop/deepMIMO/RIS/DeepMIMOv1-LIS-DeepLearning-Taha/Output_Python/Neural_Network/tensorboard_logs --port=6006
%tensorboard --logdir=/mnt/c/Users/Work/Desktop/deepMIMO/RIS/DeepMIMOv1-LIS-DeepLearning-Taha/Output_Python/Neural_Network/tensorboard_logs --port=6006 --host=localhost

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 67244), started 0:07:53 ago. (Use '!kill 67244' to kill it.)

In [48]:
# ------------------ Training Options ------------------ #
max_epochs = 20
mini_batch_size = 500

#if Training_Size_dd < mini_batch_size:
#    validationFrequency = Training_Size_dd
#else:
#    validationFrequency = int(np.floor(Training_Size_dd/mini_batch_size))
validationFrequency = 1
        
# ------------------ DL Model Training ------------------ #
#train_dataset = tf.data.Dataset.from_tensor_slices((x, Y_train)).batch(mini_batch_size).prefetch(tf.data.AUTOTUNE)
#val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val)).batch(mini_batch_size).prefetch(tf.data.AUTOTUNE)

print("Start DL training...")
start_time = time.time()

history = model_py.fit(
    xt, Y_train,
    validation_data=(xv, Y_val),
    #train_dataset, 
    #validation_data=val_dataset
    batch_size=mini_batch_size,
    epochs=max_epochs,
    shuffle=True,  # Shuffle data at each epoch
    callbacks=[lr_scheduler, tensorboard_callback],
    validation_freq=validationFrequency,
    verbose=2
)

elapsed_time = time.time() - start_time
print(f"Training completed in {elapsed_time / 60:.2f} minutes.")

Start DL training...
Epoch 1/20
20/20 - 3s - 126ms/step - loss: 3.0244 - mse: 0.0041 - val_loss: 2.5590 - val_mse: 0.0032 - learning_rate: 0.0250
Epoch 2/20
20/20 - 1s - 70ms/step - loss: 3.0051 - mse: 0.0041 - val_loss: 2.5249 - val_mse: 0.0032 - learning_rate: 0.0250
Epoch 3/20

Epoch 3: ReduceLROnPlateau reducing learning rate to 0.012500000186264515.
20/20 - 1s - 58ms/step - loss: 2.9878 - mse: 0.0041 - val_loss: 2.5031 - val_mse: 0.0031 - learning_rate: 0.0250
Epoch 4/20
20/20 - 2s - 77ms/step - loss: 2.9608 - mse: 0.0040 - val_loss: 2.5089 - val_mse: 0.0031 - learning_rate: 0.0125
Epoch 5/20

Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0062500000931322575.
20/20 - 1s - 60ms/step - loss: 2.9403 - mse: 0.0040 - val_loss: 2.4814 - val_mse: 0.0031 - learning_rate: 0.0125
Epoch 6/20
20/20 - 2s - 85ms/step - loss: 2.9124 - mse: 0.0039 - val_loss: 2.4689 - val_mse: 0.0031 - learning_rate: 0.0063
Epoch 7/20

Epoch 7: ReduceLROnPlateau reducing learning rate to 0.00312500004656

In [45]:
# Save the trained model
model_py.save(saved_models + 'model_' + model_type + '_' + str(Training_Size_dd) + '.keras')  # The file needs to end with the .keras extension

print(f"Model saved in {saved_models}")

#np.save(os.path.join(output_folder, 'history.npy'), history.history)
#np.save(os.path.join(output_folder, 'Y_predicted.npy'), Y_predicted)
#print("History and Y_predicted saved successfully.")

Model saved in /mnt/c/Users/Work/Desktop/deepMIMO/RIS/DeepMIMOv1-LIS-DeepLearning-Taha/Output_Python/Neural_Network/saved_models/


## DL Model Prediction

# SOSTITUIRE X_val CON X_test!!!

In [51]:
sanity = 1

In [52]:
if sanity == 1:
    
    # Sanity check modello matlab importato in python

    print(xv.shape)  # (6200, 1024)

    xv_reshaped = np.reshape(xv, (xv.shape[0], 1, 1, xv.shape[1]))
    print(xv_reshaped.shape)  # Output: (6200, 1, 1, 1024)

    X_val_reshaped = tf.transpose(xv_reshaped, perm=[0, 3, 1, 2]) 
    print(X_val_reshaped.shape)  # (6200, 1024, 1, 1)

    # Esegui predizione con DL_input_reshaped
    YPredicted_mat = model_mat.predict(X_val_reshaped, batch_size=128, verbose=1)
    print(YPredicted_mat.shape)

    Indmax_DL_mat = np.argmax(YPredicted_mat, axis=1)
    print(Indmax_DL_mat.shape)
    print(np.min(Indmax_DL_mat))
    print(np.max(Indmax_DL_mat))

    Indmax_DL = Indmax_DL_mat

(6200, 1024)
(6200, 1, 1, 1024)
(6200, 1024, 1, 1)


/home/work_wsl/miniconda3/envs/deepmimo/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_unnormalized']
Received: inputs=Tensor(shape=(128, 1024, 1, 1))
  warnings.warn(msg)


44/49 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step

/home/work_wsl/miniconda3/envs/deepmimo/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_unnormalized']
Received: inputs=Tensor(shape=(None, 1024, 1, 1))
  warnings.warn(msg)


49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step
(6200, 1024)
(6200,)
503
542


In [49]:
print("Start DL prediction...")

print(xv.shape)  # (6200, 1024)

YPredicted = model_py.predict(xv, verbose=1, batch_size=128)

print(YPredicted.shape)

Indmax_DL_py = np.argmax(YPredicted, axis=1)
print(Indmax_DL_py.shape)
print(np.min(Indmax_DL_py))
print(np.max(Indmax_DL_py))

# Questi devono essere numeri interi

Indmax_DL = Indmax_DL_py

Start DL prediction...
(6200, 1024)
 1/49 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step

49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
(6200, 1024)
(6200,)
501
543


In [53]:
# Inizializzazione
#validation_accuracy = 0
MaxR_DL = np.zeros((Indmax_DL.shape[0],), dtype=np.float32)
#MaxR_OPT = np.zeros((len(Indmax_OPT),), dtype=np.float32)

# Ciclo di confronto
for b in range(Indmax_DL.shape[0]):
    #MaxR_DL[b] = YValidation_un[b, Indmax_DL[b], 0, 0]
    MaxR_DL[b] = YValidation_un2[0, 0, Indmax_DL[b], b]
    #MaxR_OPT[b] = YValidation_un[b, Indmax_OPT[b], 0, 0]

    #if MaxR_DL[b] == MaxR_OPT[b]:
    #    validation_accuracy += 1


Rate_DL = MaxR_DL.mean()
#Rate_OPT = MaxR_OPT.mean()
#validation_accuracy = validation_accuracy / Indmax_DL.shape[0]

# Output finali
print(f"size(MaxR_DL): {MaxR_DL.shape}")
#print(f"size(MaxR_OPT): {MaxR_OPT.shape}")
#print(f"Rate_OPT: {Rate_OPT}")
print(f"Rate_DL: {Rate_DL}") # deve venire circa 1 per 10000 con 20 epoche
#print(f"validation_accuracy: {validation_accuracy}")
print("")

size(MaxR_DL): (6200,)
Rate_DL: 1.0489205121994019



In [13]:
filename_Rate_OPT = (
    f"{network_folder_in}Rate_OPT"
    f"_seed{seed}"
    f"_grid{Ur_rows[1]}"  # Ur_rows(2) in MATLAB diventa Ur_rows[1] in Python
    f"_M{My}{Mz}"
    f"_Mbar{M_bar}"
    f"_{Training_Size_dd}.mat"
)

with h5py.File(filename_Rate_OPT, 'r') as f:
    Rate_OPT = f['Rate_OPT'][:][0][0]

print(f"Rate_OPT: {Rate_OPT}")

Rate_OPT: 1.7807620763778687


### Test: import Matlab weights on Python model

In [ ]:
filename_traininfo = (
    f"{network_folder_in}traininfo"
    f"_seed{seed}"
    f"_grid{Ur_rows[1]}"
    f"_M{My}{Mz}"
    f"_Mbar{M_bar}"
    f"_{Training_Size_dd}.mat"
)

def read_mat_struct_field(file, struct_path, field):
    """Legge un campo da una struct MATLAB v7.3 (HDF5)"""
    ref = file[struct_path][field][0, 0]
    return ref

with h5py.File(filename_traininfo, 'r') as f:
    # Visualizza tutti i gruppi disponibili nella root del file
    print("Gruppi principali nel file:")
    for group in f.keys():
        print("-", group)
    
    # Visualizza le chiavi di 'traininfo'
    print("\nCampi disponibili in 'traininfo':")
    for field in f['traininfo'].keys():
        print("-", field)

    
    FinalValidationLoss = f['traininfo']['FinalValidationLoss'][0, 0]
    FinalValidationRMSE = f['traininfo']['FinalValidationRMSE'][0, 0]
    TrainingLoss = f['traininfo']['TrainingLoss'][-1, 0]
    TrainingRMSE = f['traininfo']['TrainingRMSE'][-1, 0]
    ValidationLoss = f['traininfo']['ValidationLoss'][-1, 0]
    ValidationRMSE = f['traininfo']['ValidationRMSE'][-1, 0]
    #OutputNetworkIteration = f['traininfo']['OutputNetworkIteration'][0, 0]

    # è un numero senza unità e serve principalmente come obiettivo di ottimizzazione.
    # è una misura dell'errore medio delle predizioni rispetto ai veri valori
    print("\nTrainingLoss:", TrainingLoss) 
    print("ValidationLoss:", ValidationLoss)
    print("FinalValidationLoss:", FinalValidationLoss) #  il valore finale della funzione di perdita (loss, MSE) calcolata sui dati di validazione,
    print("\nTrainingRMSE:", TrainingRMSE)
    print("ValidationRMSE:", ValidationRMSE)
    print("FinalValidationRMSE:", FinalValidationRMSE) # il valore finale del Root Mean Squared Error (RMSE) sui dati di validazione,
    #print("OutputNetworkIteration:", OutputNetworkIteration)

In [ ]:
from tensorflow.keras import Model, Input

# Definizione del nuovo input (non normalizzato)
#input_raw = Input(shape=(1, 1, X_train.shape[1]), name='input_unnormalized')
input_raw = Input(shape=(X_train.shape[1],), name='input_unnormalized')

# Ricostruzione della rete, escluso il layer 'input_'
#x = Reshape((1, 1, 1024), name='Fully1_preFlatten1')(input_raw)
x = Dense(1024, name='Fully1_')(input_raw)
x = ReLU(name='re_lu')(x)
x = Dropout(0.5, name='dropout')(x)

#x = Reshape((1, 1, 1024), name='Fully2_preFlatten1')(x)
x = Dense(4096, name='Fully2_')(x)
x = ReLU(name='re_lu_1')(x)
x = Dropout(0.5, name='dropout_1')(x)

#x = Reshape((1, 1, 4096), name='Fully3_preFlatten1')(x)
x = Dense(4096, name='Fully3_')(x)
x = ReLU(name='re_lu_2')(x)
x = Dropout(0.5, name='dropout_2')(x)

#x = Reshape((1, 1, 4096), name='Fully4_preFlatten1')(x)
#x = Dense(1024, name='Fully4_')(x)
#output = Flatten(name='flatten')(x)
output = Dense(1024, name='Fully4_')(x)

# Costruzione del nuovo modello
model_mat_no_norm = Model(inputs=input_raw, outputs=output, name='model_mat_no_norm')

# Copia dei pesi dai layer omologhi di model_mat
for layer in model_py.layers:
    try:
        corresponding_layer = model_mat_no_norm.get_layer(layer.name)
        layer.set_weights(corresponding_layer.get_weights())
        print(f"Pesi copiati per il layer: {layer.name}")
    except ValueError:
        print(f"Layer '{layer.name}' non trovato in model_mat o senza pesi.")

model_py.summary()

# Sanity check

#print(xv.shape)  # (6200, 1, 1, 1024)

#X_val_reshaped = tf.transpose(X_val, perm=[0, 3, 1, 2]) 
#print(X_val_reshaped.shape)  # (6200, 1024, 1, 1)

# Esegui predizione con DL_input_reshaped
#YPredicted_mat = model_mat_no_norm.predict(X_val_normalized, batch_size=128, verbose=1)
YPredicted_mat = model_mat_no_norm.predict(X_val, batch_size=128, verbose=1)
print(YPredicted_mat.shape)

Indmax_DL_mat_no_norm = np.argmax(YPredicted_mat, axis=1)
print(Indmax_DL_mat_no_norm.shape)
print(np.min(Indmax_DL_mat_no_norm))
print(np.max(Indmax_DL_mat_no_norm))

Indmax_DL = Indmax_DL_mat_no_norm
